In [1]:
library(tximport)
library(DESeq2)
library(apeglm)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: MatrixGe

In [2]:
dir <- "../data"
list.files(dir)

[1] "accession_log.txt"      "age-down_astrocyte.txt" "age-down_microglia.txt"
 [4] "age-up_astrocyte.txt"   "age-up_microglia.txt"   "failed_acc.txt"        
 [7] "fastq"                  "filtered"               "filtered_logs"         
[10] "metadata.txt"           "NexteraPE-PE.fa"        "Pan_sequences.fa"      
[13] "README.md"              "rsem"                   "sample_info.csv"       
[16] "SRR_Acc_List.txt"       "STAR"

In [3]:
samples <- read.table(file.path(dir, "metadata.txt"), sep = ",", header = TRUE)
head(samples)

,Run,age,Assay.Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center.Name,Consent,⋯,LibraryLayout,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,Sample.Name,source_name,SRA.Study,Strain
,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,SRR10079579,2-month-old,RNA-Seq,300,8062648800,PRJNA564299,SAMN12708617,2635498394,GEO,public,⋯,PAIRED,cDNA,TRANSCRIPTOMIC,Mus musculus,ILLUMINA,2020-04-21T00:00:00Z,GSM4065891,Brain,SRP220678,C57BL/6
2,SRR10079580,2-month-old,RNA-Seq,300,8013847800,PRJNA564299,SAMN12708616,2636901025,GEO,public,⋯,PAIRED,cDNA,TRANSCRIPTOMIC,Mus musculus,ILLUMINA,2020-04-21T00:00:00Z,GSM4065892,Brain,SRP220678,C57BL/6
3,SRR10079581,2-month-old,RNA-Seq,300,8055843600,PRJNA564299,SAMN12708615,2642059362,GEO,public,⋯,PAIRED,cDNA,TRANSCRIPTOMIC,Mus musculus,ILLUMINA,2020-04-21T00:00:00Z,GSM4065893,Brain,SRP220678,C57BL/6
4,SRR10079582,4-month-old,RNA-Seq,300,7608938400,PRJNA564299,SAMN12708614,2465813362,GEO,public,⋯,PAIRED,cDNA,TRANSCRIPTOMIC,Mus musculus,ILLUMINA,2020-04-21T00:00:00Z,GSM4065894,Brain,SRP220678,C57BL/6
5,SRR10079583,4-month-old,RNA-Seq,300,7912941900,PRJNA564299,SAMN12708613,2604058160,GEO,public,⋯,PAIRED,cDNA,TRANSCRIPTOMIC,Mus musculus,ILLUMINA,2020-04-21T00:00:00Z,GSM4065895,Brain,SRP220678,C57BL/6
6,SRR10079584,4-month-old,RNA-Seq,300,7563385200,PRJNA564299,SAMN12708612,2455545524,GEO,public,⋯,PAIRED,cDNA,TRANSCRIPTOMIC,Mus musculus,ILLUMINA,2020-04-21T00:00:00Z,GSM4065896,Brain,SRP220678,C57BL/6


In [4]:
files <- file.path(dir, "rsem", samples$Run, paste0(samples$Run, ".genes.results"))
names(files) <- paste0("sample", 1:60)
txi <- tximport(files, type = "rsem", txIn = FALSE, txOut = FALSE)
head(txi$counts)

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 
56 
57 
58 
59 
60 




,sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample8,sample9,sample10,⋯,sample51,sample52,sample53,sample54,sample55,sample56,sample57,sample58,sample59,sample60
0610005C13Rik,0,0,1,1,1,0,1,0,1,0,⋯,2,7,6,3,2,0,0,9,6,4
0610006L08Rik,0,0,0,0,0,0,0,0,0,0,⋯,0,1,0,1,0,0,0,0,0,0
0610007P14Rik,327,360,442,358,429,347,572,585,697,408,⋯,1214,1667,1672,1704,972,1167,1294,1855,1509,1333
0610009B22Rik,631,604,834,471,614,497,926,957,929,752,⋯,735,1006,1045,1054,801,901,855,1180,1002,895
0610009E02Rik,2,2,2,3,4,4,1,5,5,0,⋯,22,12,28,18,11,7,6,22,21,7
0610009L18Rik,32,64,34,48,57,28,71,53,62,48,⋯,148,117,194,88,93,114,128,184,187,125


In [5]:
# WT microglia samples: SRR10079579 - SRR10079593
micro <- txi$counts[,1:15]
head(micro)

# WT astrocyte samples: SRR10079609 - SRR10079623
astro <- txi$counts[,31:45]
head(astro)

,sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample8,sample9,sample10,sample11,sample12,sample13,sample14,sample15
0610005C13Rik,0,0,1,1,1,0,1,0,1,0,1,0,0,0,0
0610006L08Rik,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0610007P14Rik,327,360,442,358,429,347,572,585,697,408,357,412,464,511,195
0610009B22Rik,631,604,834,471,614,497,926,957,929,752,675,679,518,718,2862
0610009E02Rik,2,2,2,3,4,4,1,5,5,0,0,2,3,0,0
0610009L18Rik,32,64,34,48,57,28,71,53,62,48,38,54,25,26,0


,sample31,sample32,sample33,sample34,sample35,sample36,sample37,sample38,sample39,sample40,sample41,sample42,sample43,sample44,sample45
0610005C13Rik,0,1,0,7,7,5,6,3,3,3,3,1,7,4,3
0610006L08Rik,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0610007P14Rik,1139,1211,1403,1222,1087,1506,1987,2075,2088,1180,1216,1430,1341,1548,1378
0610009B22Rik,840,873,1007,736,691,901,1454,1432,1414,893,989,997,684,722,772
0610009E02Rik,10,4,2,17,6,19,29,27,28,2,6,8,20,16,15
0610009L18Rik,75,59,72,129,91,104,168,177,195,119,119,132,149,131,105


In [6]:
month <- factor(rep(c("2mo", "4mo", "6mo", "9mo", "12mo"), each = 3))
# Change levels so that 2mo is the reference
month <- factor(month, levels = c("2mo", "4mo", "6mo", "9mo", "12mo" )) 

# microglia colData
labelsM <- data.frame(conditionM = month, row.names = colnames(micro))
labelsM

# astrocyte colData
labelsA <- data.frame(conditionA = month, row.names = colnames(astro))
labelsA

,conditionM
,<fct>
sample1,2mo
sample2,2mo
sample3,2mo
sample4,4mo
sample5,4mo
sample6,4mo
sample7,6mo
sample8,6mo
sample9,6mo


,conditionA
,<fct>
sample31,2mo
sample32,2mo
sample33,2mo
sample34,4mo
sample35,4mo
sample36,4mo
sample37,6mo
sample38,6mo
sample39,6mo


In [7]:
# Run DESeq() on the microglia data
ddsM <- DESeqDataSetFromMatrix(countData = round(micro), colData = labelsM, design = ~ conditionM)
ddsM <- DESeq(ddsM)
resultsNames(ddsM) # gives coefficients

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Intercept"              "conditionM_4mo_vs_2mo"  "conditionM_6mo_vs_2mo" 
[4] "conditionM_9mo_vs_2mo"  "conditionM_12mo_vs_2mo"

In [8]:
# Get the results for each of the 4 microglia comparisons (with log fold change shrunken) 
res4M <- lfcShrink(ddsM, coef="conditionM_4mo_vs_2mo", type="apeglm")
res6M <- lfcShrink(ddsM, coef="conditionM_6mo_vs_2mo", type="apeglm")
res9M <- lfcShrink(ddsM, coef="conditionM_9mo_vs_2mo", type="apeglm")
res12M <- lfcShrink(ddsM, coef="conditionM_12mo_vs_2mo", type="apeglm")

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    

In [9]:
# define filtering functions
# DEGs defined as genes with padj < 0.05 and |log2FoldChange| > 0.5

# filter by pre-defined adjusted p-value and log2FoldChange
# Input: DataFrame m of DESeq2 results
# Output: filtered DataFrame containing only upregulated DEGs

filterUp <- function(m) {
    m_filt = subset(m, padj != "NA" & padj < 0.05)
    m_filt = subset(m_filt, log2FoldChange != "NA" & log2FoldChange > 0.5)
    return (m_filt)
}

# filter by pre-defined adjusted p-value and log2FoldChange
# Input: DataFrame m of DESeq2 results
# Output: filtered DataFrame containing only downregulated DEGs

filterDown <- function(m) {
    m_filt = subset(m, padj != "NA" & padj < 0.05)
    m_filt = subset(m_filt, log2FoldChange != "NA" & log2FoldChange < -0.5)
    return (m_filt)
}

In [10]:
# define function to find all DEGs
# find the DEGs that occur at each of 4 time points relative to the control 
# Input: 4 DataFrames filtered for DEGs
# Output: list of DEGs found in all DataFrames

findAllDEGs <- function(a, b, c, d) {
    # get the DEGs for each point
    DEG_a <- row.names(a)
    DEG_b <- row.names(b)
    DEG_c <- row.names(c)
    DEG_d <- row.names(d)

    # find intersection of each set of DEGs
    DEG_all <- Reduce(intersect, list(DEG_a, DEG_b, DEG_c, DEG_d))
    return (DEG_all)
}

In [11]:
# define Jaccard Similarity function
# Input: two objects of the same type containing sequence of elements
# Output: Jaccard similarity of two objects

jaccard <- function(a, b) {
    intersection = length(intersect(a, b))
    union = length(a) + length(b) - intersection
    return (intersection/union)
}

In [12]:
# filter for upregulated microglia DEGs
res4M_up <- filterUp(res4M)
res6M_up <- filterUp(res6M)
res9M_up <- filterUp(res9M)
res12M_up <- filterUp(res12M)

# filter for downregulated microglia DEGs
res4M_down <- filterDown(res4M)
res6M_down <- filterDown(res6M)
res9M_down <- filterDown(res9M)
res12M_down <- filterDown(res12M)

# view DataFrames
res4M_up
res4M_down

log2 fold change (MAP): conditionM 4mo vs 2mo 
Wald test p-value: conditionM 4mo vs 2mo 
DataFrame with 52 rows and 5 columns
               baseMean log2FoldChange     lfcSE      pvalue        padj
              <numeric>      <numeric> <numeric>   <numeric>   <numeric>
2010107G23Rik   70.4262       5.592144  0.818035 1.10732e-12 8.33587e-09
4930522L14Rik  229.2585       0.788272  0.202055 4.17571e-06 2.41806e-03
ATP8           178.2856       0.855979  0.272844 6.48576e-05 1.90161e-02
Adgrl4          28.5871       2.290025  0.776773 1.00593e-04 2.40402e-02
Anp32a         792.9677       1.101585  0.252435 5.31547e-07 6.15613e-04
...                 ...            ...       ...         ...         ...
Wfdc1           14.9516       2.972623  1.125206 2.34198e-04 4.22278e-02
Zcchc9         242.9688       0.547850  0.197283 2.30004e-04 4.22278e-02
Zfp69          964.6476       0.587253  0.113520 1.24530e-08 2.67846e-05
Zfp738         224.5707       0.817991  0.266217 8.28068e-05 2.14955e-0

log2 fold change (MAP): conditionM 4mo vs 2mo 
Wald test p-value: conditionM 4mo vs 2mo 
DataFrame with 9 rows and 5 columns
          baseMean log2FoldChange     lfcSE      pvalue        padj
         <numeric>      <numeric> <numeric>   <numeric>   <numeric>
Arl6ip5    734.634      -0.542116  0.154973 2.07678e-05 0.008685555
Atp6v0a2  1758.028      -0.537203  0.116267 1.91331e-07 0.000261880
Chst15     213.261      -0.626882  0.205655 8.74761e-05 0.021950659
Inppl1     252.577      -0.689884  0.216327 5.81593e-05 0.017826593
Map2k4     603.701      -0.504705  0.167122 1.15795e-04 0.026415365
Midn       150.573      -0.961564  0.338710 1.51416e-04 0.033525307
Nectin2    629.263      -1.150555  0.283258 2.16319e-06 0.001418505
Pa2g4      584.239      -0.738196  0.173216 9.17890e-07 0.000791849
Ptpn23     246.332      -0.615653  0.180627 3.05787e-05 0.010961739

In [24]:
# get number of microglia DEGs at each time point: upregulated + downregulated = total
# get number of upregulated microglia DEGs at each time point
nrow(res4M_up)
nrow(res6M_up)
nrow(res9M_up)
nrow(res12M_up)

# get number of downregulated microglia DEGs at each time point
nrow(res4M_down)
nrow(res6M_down)
nrow(res9M_down)
nrow(res12M_down)

[1] 52

[1] 513

[1] 101

[1] 892

[1] 9

[1] 1225

[1] 20

[1] 367

In [13]:
# find the final DEG list for each category: upregulated and downregulated
DEG_upM <- findAllDEGs(res4M_up, res6M_up, res9M_up, res12M_up)
DEG_downM <- findAllDEGs(res4M_down, res6M_down, res9M_down, res12M_down)

length(DEG_upM) 
length(DEG_downM) 

[1] 4

[1] 0

In [14]:
# Compare my DEG list to paper's list

# read in paper's microglia DEG lists: all genes listed under column "V1"
age_upM <- read.table(file.path(dir, "age-up_microglia.txt")) 
age_downM <- read.table(file.path(dir, "age-down_microglia.txt")) 

# calculate Jaccard similarity for upregulated microglia DEG lists
upM_jaccard <- jaccard(DEG_upM, age_upM$V1)
upM_jaccard

# calculate Jaccard similarity for downregulated microglia DEG lists
downM_jaccard <- jaccard(DEG_downM, age_downM$V1)
downM_jaccard

[1] 0.06122449

[1] 0

In [15]:
# Run DESeq() on the astrocyte data
ddsA <- DESeqDataSetFromMatrix(countData = round(astro), colData = labelsA, design = ~ conditionA)
ddsA <- DESeq(ddsA)
resultsNames(ddsA) # gives coefficients

converting counts to integer mode

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



[1] "Intercept"              "conditionA_4mo_vs_2mo"  "conditionA_6mo_vs_2mo" 
[4] "conditionA_9mo_vs_2mo"  "conditionA_12mo_vs_2mo"

In [16]:
# Get the results for each of the 4 astrocyte comparisons (with log fold change shrunken) 
res4A <- lfcShrink(ddsA, coef="conditionA_4mo_vs_2mo", type="apeglm")
res6A <- lfcShrink(ddsA, coef="conditionA_6mo_vs_2mo", type="apeglm")
res9A <- lfcShrink(ddsA, coef="conditionA_9mo_vs_2mo", type="apeglm")
res12A <- lfcShrink(ddsA, coef="conditionA_12mo_vs_2mo", type="apeglm")

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    

In [17]:
# filter for upregulated astrocyte DEGs
res4A_up <- filterUp(res4A)
res6A_up <- filterUp(res6A)
res9A_up <- filterUp(res9A)
res12A_up <- filterUp(res12A)

# filter for downregulated astrocyte DEGs
res4A_down <- filterDown(res4A)
res6A_down <- filterDown(res6A)
res9A_down <- filterDown(res9A)
res12A_down <- filterDown(res12A)

# view DataFrames
res4A_up
res4A_down

log2 fold change (MAP): conditionA 4mo vs 2mo 
Wald test p-value: conditionA 4mo vs 2mo 
DataFrame with 842 rows and 5 columns
               baseMean log2FoldChange     lfcSE      pvalue        padj
              <numeric>      <numeric> <numeric>   <numeric>   <numeric>
0610011F06Rik  466.9239       0.669540  0.121530 3.76725e-09 3.26299e-07
1110001J03Rik 1064.3740       0.829392  0.132706 3.67071e-11 5.40212e-09
1110020A21Rik   83.1407       0.919341  0.272069 4.69178e-05 1.02260e-03
1500015L24Rik  190.2579       0.547403  0.280360 4.41965e-03 3.32273e-02
1500015O10Rik  753.9605       0.957601  0.229835 1.98084e-06 7.56354e-05
...                 ...            ...       ...         ...         ...
Zfp97           99.4503       0.808682  0.290427 3.42868e-04 4.86925e-03
Zfp971         155.9585       0.770504  0.182782 2.11024e-06 7.95767e-05
Zfp973          14.1934       3.345328  1.017625 6.00508e-05 1.23442e-03
Zhx3           415.2115       1.030687  0.189755 3.65205e-09 3.19650e-

log2 fold change (MAP): conditionA 4mo vs 2mo 
Wald test p-value: conditionA 4mo vs 2mo 
DataFrame with 595 rows and 5 columns
               baseMean log2FoldChange     lfcSE      pvalue        padj
              <numeric>      <numeric> <numeric>   <numeric>   <numeric>
1810041L15Rik  138.4930      -0.635180  0.249507 8.93452e-04 1.02651e-02
2610002M06Rik  243.8672      -0.584906  0.246201 1.56182e-03 1.56182e-02
3110035E14Rik   43.7788      -1.883770  0.348049 3.38556e-09 2.99478e-07
4930452G13Rik   80.0485      -0.671214  0.290098 1.45594e-03 1.48087e-02
4933431E20Rik 1363.7916      -0.784051  0.123775 1.99457e-11 3.12921e-09
...                 ...            ...       ...         ...         ...
Xkr4           103.4972      -0.895074  0.240946 1.32341e-05 3.69888e-04
Zdhhc2         151.5495      -1.180639  0.202745 3.43567e-10 4.05214e-08
Zdhhc9        3258.2406      -1.121037  0.194499 4.94459e-10 5.53124e-08
Zfp423         249.9800      -0.593120  0.164663 3.61974e-05 8.37223e-

In [25]:
# get number of astrocyte DEGs at each time point: upregulated + downregulated = total
# get number of upregulated astrocyte DEGs at each time point
nrow(res4A_up)
nrow(res6A_up)
nrow(res9A_up)
nrow(res12A_up)

# get number of downregulated astrocyte microglia DEGs at each time point
nrow(res4A_down)
nrow(res6A_down)
nrow(res9A_down)
nrow(res12A_down)

[1] 842

[1] 3426

[1] 962

[1] 1555

[1] 595

[1] 3863

[1] 1195

[1] 1610

In [18]:
# find the final DEG list for each category: upregulated and downregulated
DEG_upA <- findAllDEGs(res4A_up, res6A_up, res9A_up, res12A_up)
DEG_downA <- findAllDEGs(res4A_down, res6A_down, res9A_down, res12A_down)

length(DEG_upA) 
length(DEG_downA)

[1] 216

[1] 176

In [19]:
# Compare my DEG list to paper's list

# read in paper's astrocyte DEG lists: all genes listed under column "V1"
age_upA <- read.table(file.path(dir, "age-up_astrocyte.txt")) 
age_downA <- read.table(file.path(dir, "age-down_astrocyte.txt")) 

# calculate Jaccard similarity for upregulated astrocyte DEG lists
upA_jaccard <- jaccard(DEG_upA, age_upA$V1)
upA_jaccard

# calculate Jaccard similarity for downregulated astrocyte DEG lists
downA_jaccard <- jaccard(DEG_downA, age_downA$V1)
downA_jaccard

[1] 0.4300699

[1] 0.6355556